In [ ]:
import pandas as pd

df_mortalitat = pd.read_csv('C:/Users/David/Desktop/Analitica/Datasets/sql_mortalidad_recetas/Datasets/Mortalitat_Catalunya.csv', delimiter=',')


In [ ]:
df_receptes = pd.read_csv("C:/Users/David/Desktop/Analitica/Datasets/sql_mortalidad_recetas/Datasets/Receptes_facturades.csv", nrows=10000, delimiter = ',')

In [4]:
# # Info columnas
# df_mortalitat.info()
# df_receptes.info()

# En python no se han inducido los tipos de datos bien 

#HAGO QUE PANDAS REEVALUE EL ***TIPO DE DATOS*** ÓPTIMO PARA CADA COLUMNA
df_mortalitat = df_mortalitat.convert_dtypes()
df_receptes = df_receptes.convert_dtypes()
print(df_receptes.dtypes)

#Consulto de nuevo. Veo que han mejorado.
# df_mortalitat.info()
# df_receptes.info()

# ***NOMBRES DE COLUMNA***
# Encabezados en catalan y algunos no muy claros
# Aclarar los encabezados y los tipos de datos de cada columna.
df_mortalitat = df_mortalitat.rename(columns={
    "Sexe": "sexo",
    "Edat": "edad",
    "Data": "fecha",
    "Defuncions observades": "defunciones_observadas",
    "Defuncions esperades": "defunciones_esperadas",
    "Defuncions esperades (Límit superior)": "defunciones_esperadas_limite_inf",
    "Defuncions esperades (Límit inferior)": "defunciones_esperadas_limite_sup"
})

df_receptes = df_receptes.rename(columns={
    "any":"anho",
    "codi de la regió sanitària": "codigo_reg_sanitaria",
    "regió sanitària":"region_sanitaria",
    "grup d'edat":"grupo_edad",
    "sexe":"sexo",
    "codi del grup ATC nivell 1":"codigo_grupo_ATC_nv_1",
    "grup ATC nivell 1":"grupo_ATC_nv_1",
    "codi del grup ATC nivell 2":"codigo_grupo_ATC_nv_2",
    "grup ATC nivell 2":"grupo_ATC_nv_2",
    "codi del grup ATC nivell 3":"codigo_grupo_ATC_nv_3",
    "grup ATC nivell 3":"grupo_ATC_nv_3",
    "codi del grup ATC nivell 4":"codigo_grupo_ATC_nv_4",
    "grup ATC nivell 4":"grupo_ATC_nv_4",
    "nombre de receptes":"numero_receta",
    "nombre d'envasos":"numero_envase",
    "import íntegre":"importe_integro",
    "import aportació CatSalut":"importe_aportacion_cat_salut"
})

any                                    Int64
mes                                    Int64
codi de la regió sanitària             Int64
regió sanitària               string[python]
grup d'edat                   string[python]
sexe                          string[python]
codi del grup ATC nivell 1    string[python]
grup ATC nivell 1             string[python]
codi del grup ATC nivell 2    string[python]
grup ATC nivell 2             string[python]
codi del grup ATC nivell 3    string[python]
grup ATC nivell 3             string[python]
codi del grup ATC nivell 4    string[python]
grup ATC nivell 4             string[python]
nombre de receptes                     Int64
nombre d'envasos                       Int64
import íntegre                       Float64
import aportació CatSalut            Float64
dtype: object


In [5]:
# ***CAMPOS FECHA***

# !!!!!! Como en mi string está en formato español el dia está primero.
# Tengo que indicar correctamente si dayfirst = True. Pandas puede equivocarse.
df_mortalitat["fecha"] = pd.to_datetime(df_mortalitat["fecha"], errors='coerce', dayfirst=True)

#En el caso de las recetas me cargo las 2 columnas anho, mes
df_receptes['fecha'] = pd.to_datetime(df_receptes['anho'].astype(str) + '-' + df_receptes['mes'].astype(str).str.zfill(2) + '-01')
df_receptes = df_receptes.drop(columns=['anho','mes'])

# Reordeno columnas
cols = df_receptes.columns.tolist()
cols.insert(0, cols.pop(cols.index('fecha')))
df_receptes = df_receptes[cols]


In [6]:
# ***LIMPIEZA DATOS*** - Traduccion columnas sexo
df_mortalitat['sexo'] = df_mortalitat['sexo'].str.replace('dones', 'mujer', regex=False)
df_mortalitat['sexo'] = df_mortalitat['sexo'].str.replace('homes', 'hombre', regex=False)
df_mortalitat['sexo'] = df_mortalitat['sexo'].str.replace('tots', 'ambos', regex=False)

df_receptes['sexo'] = df_receptes['sexo'].str.replace('Dona', 'mujer', regex=False)
df_receptes['sexo'] = df_receptes['sexo'].str.replace('Home', 'hombre', regex=False)
df_receptes['sexo'] = df_receptes['sexo'].str.replace('Dona', 'mujer', regex=False)

# ***LIMPIEZA DATOS*** - Valores de Rangos de Edad
df_receptes['grupo_edad'] = df_receptes['grupo_edad'].str.replace(' anys', '', regex=False)
df_receptes['grupo_edad'] = df_receptes['grupo_edad'].str.replace(' any', '', regex=False)
df_receptes['grupo_edad'] = df_receptes['grupo_edad'].str.replace('Més de 84', '84-999', regex=False)

df_mortalitat['edad'] = df_mortalitat['edad'].str.replace('edat ', '', regex=False)
df_mortalitat['edad'] = df_mortalitat['edad'].str.replace('>= 65', '65-999', regex=False)
df_mortalitat['edad'] = df_mortalitat['edad'].str.replace('>= 85', '85-999', regex=False)
df_mortalitat['edad'] = df_mortalitat['edad'].str.replace('tots', '0-999', regex=False)

## ***LIMPIEZA DATOS*** - Creacion columnas EDAD_MIN EDAD_MAX.
# df_mortalitat.groupby('edad').size()
# df_receptes.groupby('grupo_edad').size()

## CREO COLUMNAS EDAD_MIN, EDAD_MAX para tener 2 campos para el rango de edades.
df_mortalitat[['edad_min','edad_max']] = df_mortalitat['edad'].str.split('-',expand=True)
df_receptes[['edad_min','edad_max']] = df_receptes['grupo_edad'].str.split('-',expand=True)

## Comprobacion que los grupos están correctos
# print(df_mortalitat.groupby(['edad_min', 'edad_max']).size())
# print(df_receptes.groupby(['edad_min', 'edad_max']).size())

## ***LIMPIEZA DATOS*** - Coloco columnas
df_mortalitat = df_mortalitat.sort_values(by=['edad_min','edad_max', 'fecha'], ascending=True)
df_receptes = df_receptes.sort_values(by=['edad_min','edad_max'], ascending=True)

In [7]:
# *** REEVALUAR TIPO DE DATOS DE LOS DF TRAS LIMPIAR ***

#Tras haber jugado con las columnas me cercioro del tipo de datos

#HAGO QUE PANDAS REEVALUE EL ***TIPO DE DATOS*** ÓPTIMO PARA CADA COLUMNA
df_mortalitat = df_mortalitat.convert_dtypes()
df_receptes = df_receptes.convert_dtypes()


# Corrijo tipos erroneos
# Mortalidad
df_mortalitat['edad_min'] = pd.to_numeric(df_mortalitat['edad_min'], errors='coerce').astype('Int64')
df_mortalitat['edad_max'] = pd.to_numeric(df_mortalitat['edad_max'], errors='coerce').astype('Int64')
df_mortalitat['defunciones_observadas'] = df_mortalitat['defunciones_observadas'].round().astype('Int64')

# Recetas
df_receptes['edad_min'] = pd.to_numeric(df_receptes['edad_min'], errors='coerce').astype('Int64')
df_receptes['edad_max'] = pd.to_numeric(df_receptes['edad_max'], errors='coerce').astype('Int64')

print(df_receptes.dtypes)
print(df_mortalitat.dtypes)

fecha                           datetime64[ns]
codigo_reg_sanitaria                     Int64
region_sanitaria                string[python]
grupo_edad                      string[python]
sexo                            string[python]
codigo_grupo_ATC_nv_1           string[python]
grupo_ATC_nv_1                  string[python]
codigo_grupo_ATC_nv_2           string[python]
grupo_ATC_nv_2                  string[python]
codigo_grupo_ATC_nv_3           string[python]
grupo_ATC_nv_3                  string[python]
codigo_grupo_ATC_nv_4           string[python]
grupo_ATC_nv_4                  string[python]
numero_receta                            Int64
numero_envase                            Int64
importe_integro                        Float64
importe_aportacion_cat_salut           Float64
edad_min                                 Int64
edad_max                                 Int64
dtype: object
sexo                                string[python]
edad                                string

In [8]:
## ***LIMPIEZA DATOS*** - Buscamos datos NULL
df_mortalitat.isna().sum()

sexo                                0
edad                                0
fecha                               0
defunciones_observadas              0
defunciones_esperadas               0
defunciones_esperadas_limite_sup    0
defunciones_esperadas_limite_inf    0
edad_min                            0
edad_max                            0
dtype: int64

In [8]:
## ***LIMPIEZA DATOS*** - Buscamos datos NULL
# NUESTRO ARCHIVO DE RECETAS ES DEMASIADO GRANDE 1.4GB PARA 
# LEERSE DE UNA VEZ Y ENCONTRAR NULOS. VAMOS A IR LEYENDO POR BLOQUES

# Inicializamos contador de nulos y total de filas
null_counts = None
total_counts = None

# Procesamos por chunks (ajusta el tamaño si hace falta)
for chunk in pd.read_csv("./Datasets/Receptes_facturades.csv", chunksize=100_000):
    if null_counts is None:
        null_counts = chunk.isna().sum()
        total_counts = chunk.shape[0]
    else:
        null_counts += chunk.isna().sum()
        total_counts += chunk.shape[0]

# Porcentaje de nulos por columna
null_percentages = (null_counts / total_counts) * 100

print("Nulos por columna:")
print(null_counts)

print("\nPorcentaje de nulos por columna:")
print(null_percentages)

C:\Users\David\AppData\Local\Temp\ipykernel_7156\3392572167.py:10: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("./Datasets/Receptes_facturades.csv", chunksize=100_000):
C:\Users\David\AppData\Local\Temp\ipykernel_7156\3392572167.py:10: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("./Datasets/Receptes_facturades.csv", chunksize=100_000):
C:\Users\David\AppData\Local\Temp\ipykernel_7156\3392572167.py:10: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("./Datasets/Receptes_facturades.csv", chunksize=100_000):
C:\Users\David\AppData\Local\Temp\ipykernel_7156\3392572167.py:10: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("./Datasets/Receptes_facturades.csv", chunksize=10

Nulos por columna:
any                           0
mes                           0
codi de la regió sanitària    0
regió sanitària               0
grup d'edat                   0
sexe                          0
codi del grup ATC nivell 1    0
grup ATC nivell 1             0
codi del grup ATC nivell 2    0
grup ATC nivell 2             0
codi del grup ATC nivell 3    0
grup ATC nivell 3             0
codi del grup ATC nivell 4    0
grup ATC nivell 4             0
nombre de receptes            0
nombre d'envasos              0
import íntegre                0
import aportació CatSalut     0
dtype: int64

Porcentaje de nulos por columna:
any                           0.0
mes                           0.0
codi de la regió sanitària    0.0
regió sanitària               0.0
grup d'edat                   0.0
sexe                          0.0
codi del grup ATC nivell 1    0.0
grup ATC nivell 1             0.0
codi del grup ATC nivell 2    0.0
grup ATC nivell 2             0.0
codi del grup ATC 

C:\Users\David\AppData\Local\Temp\ipykernel_7156\3392572167.py:10: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("./Datasets/Receptes_facturades.csv", chunksize=100_000):


In [9]:
## ***LIMPIEZA DE DATOS*** - campos de grupos y regiones pasan a modo Title
cols = ['region_sanitaria', 'grupo_ATC_nv_1', 'grupo_ATC_nv_2', 'grupo_ATC_nv_3', 'grupo_ATC_nv_4']
for c in cols:
    df_receptes[c] = df_receptes[c].str.title()



In [10]:
#TABLAS MORTALIDAD
# tabla mortalidad		id	fecha	id_rango_edad	id_sexo	defunciones_observadas	defunciones_esperadas	defunciones_esperadas_limite_sup	defunciones_esperadas_limite_inf
									
# tabla sexo		id_sexo	sexo_nombre sexo_abreviado						
									
#Creo tabla sexo
df_sexo = df_mortalitat[['sexo']].copy()
df_sexo = df_sexo.drop_duplicates().reset_index().drop(columns='index')
df_sexo['sexo_abreviado'] = df_sexo['sexo'].replace({'ambos':'A', 'hombre':'H', 'mujer':'M'})
df_sexo = df_sexo.rename(columns={'sexo':'sexo_nombre'})
df_sexo['id_sexo'] = df_sexo.index
df_sexo = df_sexo[['id_sexo', 'sexo_nombre','sexo_abreviado']]
df_sexo.head(10)


,id_sexo,sexo_nombre,sexo_abreviado
0,0,ambos,A
1,1,hombre,H
2,2,mujer,M


In [11]:
#Creo tabla rango_edad_amplio
# tabla rango_edad_amplio		id_rango	edad_min	edad_max			

df_rango_edad_amplio = df_mortalitat[['edad_min','edad_max']].copy()
df_rango_edad_amplio = df_rango_edad_amplio.drop_duplicates().sort_values(by=['edad_min','edad_max'], ascending=[True,True]).reset_index().drop(columns='index')
df_rango_edad_amplio['id_rango_edad_amplio'] = df_rango_edad_amplio.index
df_rango_edad_amplio = df_rango_edad_amplio[['id_rango_edad_amplio', 'edad_min','edad_max']]

df_rango_edad_amplio.head(10)

,id_rango_edad_amplio,edad_min,edad_max
0,0,0,14
1,1,0,999
2,2,15,44
3,3,45,64
4,4,65,74
5,5,65,999
6,6,75,84
7,7,85,999


In [12]:
# Creo tabla mortalidad final para la BD
# tabla mortalidad		id	fecha	id_rango_edad	id_sexo	defunciones_observadas	defunciones_esperadas	defunciones_esperadas_limite_sup	defunciones_esperadas_limite_inf
df_mortalidad_BD = df_mortalitat.copy()
df_mortalidad_BD = df_mortalidad_BD.merge(df_sexo[['sexo_nombre', 'id_sexo']], left_on='sexo', right_on='sexo_nombre', how='left')
df_mortalidad_BD = df_mortalidad_BD.merge(df_rango_edad_amplio[['edad_min','edad_max', 'id_rango_edad_amplio']], on=['edad_min','edad_max'], how='left')
df_mortalidad_BD = df_mortalidad_BD.drop(columns=['sexo','edad','edad_min','edad_max', 'sexo_nombre'])
df_mortalidad_BD['id_mortalidad'] = df_mortalidad_BD.index
df_mortalidad_BD = df_mortalidad_BD[['id_mortalidad','fecha','id_sexo','id_rango_edad_amplio', 'defunciones_observadas','defunciones_esperadas', 'defunciones_esperadas_limite_sup','defunciones_esperadas_limite_inf']]
df_mortalidad_BD.head(10)

,id_mortalidad,fecha,id_sexo,id_rango_edad_amplio,defunciones_observadas,defunciones_esperadas,defunciones_esperadas_limite_sup,defunciones_esperadas_limite_inf
0,0,2015-01-01,0,0,1,0.78,0,3
1,1,2015-01-01,1,0,1,0.46,0,3
2,2,2015-01-01,2,0,0,0.32,0,2
3,3,2015-01-02,2,0,0,0.32,0,2
4,4,2015-01-02,0,0,0,0.78,0,3
5,5,2015-01-02,1,0,0,0.46,0,3
6,6,2015-01-03,2,0,0,0.32,0,2
7,7,2015-01-03,1,0,0,0.46,0,3
8,8,2015-01-03,0,0,0,0.78,0,3
9,9,2015-01-04,2,0,0,0.32,0,2


In [13]:
#TABLAS RECETAS
# tabla rango_edad_corto		id_rango_edad_corto	edad_min	edad_max							
											
# tabla region_sanitaria		id_region_sanitaria	codigo_region_sanitaria	nombre_region_sanitaria							
											
# tabla sexo (misma que antes) 		id_sexo	sexo_name								
											
# tabla_grupos_ATC		id_grupo_ATC	nivel_grupo	codigo_grupo	descripcion_grupo	id_grupo_ATC_padre					
											
# tabla_recetas		id_receta	fecha	id_region_sanitaria	id_rango_edad_corto	id_sexo	id_grupo_ATC_final	numero_receta	numero_envase	importe_integro	importe_aportacion_cat_salut


In [14]:
#Creo tabla rango_edad_corto
df_rango_edad_corto = df_receptes[['edad_min','edad_max']].copy()
df_rango_edad_corto = df_rango_edad_corto.drop_duplicates().sort_values(by=['edad_min','edad_max'], ascending=[True,True]).reset_index().drop(columns='index')
df_rango_edad_corto['id_rango_edad_corto'] = df_rango_edad_corto.index
df_rango_edad_corto = df_rango_edad_corto[['id_rango_edad_corto', 'edad_min','edad_max']]
df_rango_edad_corto.head(10)

,id_rango_edad_corto,edad_min,edad_max
0,0,0,1
1,1,2,3
2,2,4,5
3,3,6,7
4,4,8,9
5,5,10,11
6,6,12,13
7,7,14,15
8,8,16,17
9,9,18,19


In [15]:
#Creo tabla region_sanitaria
df_region_sanitaria = df_receptes[['codigo_reg_sanitaria', 'region_sanitaria']].copy()
df_region_sanitaria = df_region_sanitaria.drop_duplicates().sort_values(by=['codigo_reg_sanitaria'], ascending=True).reset_index().drop(columns='index')
df_region_sanitaria['id_region_sanitaria'] = df_region_sanitaria.index
df_region_sanitaria = df_region_sanitaria[['id_region_sanitaria', 'codigo_reg_sanitaria','region_sanitaria']]
df_region_sanitaria = df_region_sanitaria.rename(columns={'region_sanitaria':'region_sanitaria_nombre'})
df_region_sanitaria.head(10)

,id_region_sanitaria,codigo_reg_sanitaria,region_sanitaria_nombre
0,0,61,Lleida
1,1,62,Camp De Tarragona
2,2,67,Catalunya Central


In [16]:
#Creo tabla grupos_ATC
# tabla_grupos_ATC		id_grupo_ATC	nivel_grupo	descripcion_grupo	id_grupo_ATC_padre

import numpy as np

df_grupos_ATC = df_receptes[['codigo_grupo_ATC_nv_1', 'grupo_ATC_nv_1']].copy()
df_grupos_ATC['id_grupo_ATC_padre'] = np.nan
df_grupos_ATC['nivel_grupo'] = 1
df_grupos_ATC = df_grupos_ATC.rename(columns={'codigo_grupo_ATC_nv_1':'id_grupo_ATC', 'grupo_ATC_nv_1':'descripcion_grupo'})

niveles = [2,3,4]
for n in niveles:
    codigo = f'codigo_grupo_ATC_nv_{n}'
    desc_col = f'grupo_ATC_nv_{n}'
    id_padre = f'codigo_grupo_ATC_nv_{n-1}'
    

    df_temp = df_receptes[[codigo, desc_col, id_padre]].copy()

    df_temp = df_temp.rename(columns={codigo:'id_grupo_ATC', desc_col:'descripcion_grupo', id_padre:'id_grupo_ATC_padre'})
    df_temp['nivel_grupo'] = n

    df_grupos_ATC = pd.concat([df_grupos_ATC,df_temp])

df_grupos_ATC = df_grupos_ATC.drop_duplicates().sort_values(by=['nivel_grupo','id_grupo_ATC']).reset_index(drop=True)
df_grupos_ATC = df_grupos_ATC[['id_grupo_ATC', 'nivel_grupo','descripcion_grupo','id_grupo_ATC_padre']]
df_grupos_ATC.head(100)


C:\Users\David\AppData\Local\Temp\ipykernel_10320\900002013.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_grupos_ATC = pd.concat([df_grupos_ATC,df_temp])


,id_grupo_ATC,nivel_grupo,descripcion_grupo,id_grupo_ATC_padre
0,A,1,Tracto Alimentario Y Metabolismo,<NA>
1,B,1,Sangre Y Organos Hematopoyeticos,<NA>
2,C,1,Sistema Cardiovascular,<NA>
3,D,1,Dermatologicos,<NA>
4,F,1,Sense Especificar,<NA>
...,...,...,...,...
95,V07,2,Todo El Resto De Los Productos No Terapeuticos,V
96,A01A,3,Preparados Estomatologicos,A01
97,A02B,3,Agentes Contra La Ulcera Peptica Y El Reflujo ...,A02
98,A03A,3,Agentes Contra Padecimientos Funcionales Del E...,A03


In [16]:
# tabla_recetas		id_receta	fecha	id_region_sanitaria	id_rango_edad_corto	id_sexo	id_grupo_ATC_final	numero_receta	numero_envase	importe_integro	importe_aportacion_cat_salut

df_recetas_BD = df_receptes[['fecha','codigo_reg_sanitaria',	'region_sanitaria',	'grupo_edad', 'edad_min','edad_max','sexo','codigo_grupo_ATC_nv_4', 'grupo_ATC_nv_4', 'numero_receta', 'numero_envase', 'importe_integro', 'importe_aportacion_cat_salut']].copy()
df_recetas_BD = df_recetas_BD.merge(df_sexo[['sexo_nombre', 'id_sexo']], left_on='sexo', right_on='sexo_nombre', how='left')
df_recetas_BD = df_recetas_BD.merge(df_region_sanitaria[['codigo_reg_sanitaria', 'id_region_sanitaria']], on='codigo_reg_sanitaria', how='left')
df_recetas_BD = df_recetas_BD.merge(df_rango_edad_corto[['edad_min', 'edad_max','id_rango_edad_corto']], on=['edad_min','edad_max'], how='left')
df_recetas_BD = df_recetas_BD.drop(columns=['sexo_nombre', 'sexo', 'edad_min', 'edad_max', 'grupo_edad', 'region_sanitaria', 'codigo_reg_sanitaria', 'grupo_ATC_nv_4'])
df_recetas_BD = df_recetas_BD.rename(columns={'codigo_grupo_ATC_nv_4':'id_grupo_ATC'})
df_recetas_BD['id_receta'] = df_recetas_BD.index
df_recetas_BD = df_recetas_BD[['id_receta','fecha', 'id_region_sanitaria', 'id_sexo','id_rango_edad_corto', 'id_grupo_ATC', 'numero_receta', 'numero_envase','importe_integro', 'importe_aportacion_cat_salut']]

df_recetas_BD.head(20)


,id_receta,fecha,id_region_sanitaria,id_sexo,id_rango_edad_corto,id_grupo_ATC,numero_receta,numero_envase,importe_integro,importe_aportacion_cat_salut
0,0,2016-01-01,0,2,0,A01AB,15,15,42.45,26.04
1,1,2016-01-01,0,2,0,A02BC,1,1,2.19,1.31
2,2,2016-01-01,0,2,0,A03FA,2,2,5.0,3.0
3,3,2016-01-01,0,2,0,A07AA,6,6,18.72,11.22
4,4,2016-01-01,0,2,0,A07CA,40,40,108.81,67.61
5,5,2016-01-01,0,2,0,A11CC,336,336,1442.07,855.6
6,6,2016-01-01,0,2,0,B03AB,2,2,25.04,15.02
7,7,2016-01-01,0,2,0,B03BB,1,1,2.5,1.25
8,8,2016-01-01,0,2,0,B05BB,7,7,13.68,9.97
9,9,2016-01-01,0,2,0,C07AA,1,1,225.59,221.35


In [1]:
#TODAS LAS TABLAS FINALES
#MORTALIDAD
# df_sexo.head(10)
# df_rango_edad_amplio.head(10)
df_mortalidad_BD.head(10)

# print(df_sexo.dtypes) #tipo corregido
# print(df_rango_edad_amplio.dtypes) # Tipo corregido
# print(df_mortalidad_BD.dtypes) #Tipo corregido.. decimales en muertes?.... en el CSV esta asi....

#RECETAS
# df_region_sanitaria.head(10)
# df_rango_edad_corto.head(100)
# df_grupos_ATC.head(10)
# df_recetas_BD.head(10)

# print(df_region_sanitaria.dtypes)# Tipo corregido
# print(df_rango_edad_corto.dtypes)# Tipo corregido
# print(df_grupos_ATC.dtypes)# Tipo corregido
# print(df_recetas_BD.dtypes)# Tipo corregido


#--------DAVID DEL FUTURO, Todo listo para subir a postgre.---------------
#mirar cual es la manera profesional de subir los datos a la base de datos
#enterarse de como se monta la estructura profesionalmente, como se documenta
#como se documenta el esquema, si se hace en codigo SQL, que partes hay que documentar
#Como esquema entidad relacion, tablas, etc etc. 

#Enterarse como securizar datos. Esto puede ser muy útil para el trabajo.
#Enterarse como hacer backup periodicos en la base de datos.
# Enterarse como alojarla en servidor??
#Enterarse como hacer analítica una vez tienes la bd Aparte de python
# ....


NameError: name 'df_mortalidad_BD' is not defined